In [1]:
from api import spotify_api
import json
import datetime

settings_file = 'api/settings.txt'
albums_cache_file = 'albums_cache.txt'

max_album_age = 4 #years

In [2]:
access_token = spotify_api.refresh_token(settings_file)


Requesting refresh: 
<Response [200]>
200 OK
BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA
BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA


In [3]:
spotify_request_url = 'https://api.spotify.com/v1/'

next_request = spotify_request_url + 'me/following?type=artist&limit=10'
artists_dict = {}

while(True):
    next_request = next_request + '&access_token=' + access_token.strip()
    artists_json = spotify_api.spotify_request(next_request)

    print('\n')
    artist_items = artists_json['artists']['items']
    for item in artist_items:
        print(item['name']+' -> '+item['id'])
        artists_dict[item['name']] = item['id']

    next_request = artists_json['artists']['next']
    
    print('\n')
    if not next_request:
        print("---END---")
        break


Requesting: https://api.spotify.com/v1/me/following?type=artist&limit=10&access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA
200 OK


Skyharbor -> 0YbvZPP1HufDyWIsR0ei6K
The Algorithm -> 14u4KXVp0iXQil79EpxXGc
Oh Hiroshima -> 19iNYCVHcMwJMS3NZqhZ59
If These Trees Could Talk -> 2GVzsXcXyU95u2EahzwqN7
Caravan Palace -> 37J1PlAkhRK7yrZUtqaUpQ
Steel Panther -> 3l02WF362j1oHOurzuseBv
Electric Wizard -> 4htjQW3lgIwL6fEJlTOez4
In Flames -> 57ylwQTnFnIhJh4nu4rxCs
Motionless In White -> 6MwPCCR936cYfM1dLsGVnl
Godsmack -> 6gZq1Q6bdOxsUPUG1TaFbF



Requesting: https://api.spotify.com/v1/me/following?type=artist&after=6gZq1Q6bdOxsUPUG1TaFbF&limit=10&access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJO

In [7]:
albums_list = []
with open(albums_cache_file, 'r') as f:
    for album_id in f.readlines():
        albums_list.append(album_id.strip())

new_albums_dict = {}
new_album_counter = 0
current_year = datetime.datetime.now().year

with open(albums_cache_file, 'a') as f:
    for artist in artists_dict:
        artist_id = artists_dict[artist]
        print('\nArtist: ' + artist)

        next_request = spotify_request_url + 'artists/'+artist_id+'/albums?limit=50&album_type=album,single&market=PL'

        while(True):
            next_request = next_request + '&access_token=' + access_token.strip()
            albums_json = spotify_api.spotify_request(next_request)

            album_items = albums_json['items']
            for item in album_items:
                print(item['name']+' -> '+item['release_date'])
                if item['id'] not in albums_list:
                    if current_year - int(item['release_date'][:4]) < max_album_age:
                        new_albums_dict[item['id']] = (artist, item['name'], item['release_date'])
                        f.write(item['id']+'\n')
                        new_album_counter = new_album_counter + 1

            next_request = albums_json['next']
            if not next_request:
                break
                
print('New albums: ' + str(new_album_counter))


Artist: Skyharbor

Requesting: https://api.spotify.com/v1/artists/0YbvZPP1HufDyWIsR0ei6K/albums?limit=50&album_type=album,single&market=PL&access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA
200 OK
Guiding Lights -> 2014-11-10
Blinding White Noise: Illusion & Chaos -> 2012-04-23
Chemical Hands -> 2016-09-23
Blind Side -> 2016-06-09
Out of Time -> 2015-08-25
Evolution -> 2014-06-12

Artist: The Algorithm

Requesting: https://api.spotify.com/v1/artists/14u4KXVp0iXQil79EpxXGc/albums?limit=50&album_type=album,single&market=PL&access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0

In [8]:
new_albums_dict

{'2W5WbsWmqJ2ZdGHumESMhZ': ('Steel Panther',
  'Poontang Boomerang',
  '2017-02-10'),
 '3WZomucoEBQzFTZ9A7mMTz': ('Steel Panther',
  'I Got What You Want',
  '2017-03-17'),
 '3eedTLY7FmfFjiV3PCaemY': ('Steel Panther',
  "Live from Lexxi's Mom's Garage",
  '2016-02-26'),
 '5M0f3Rs2UxEYXm5C6EaHvG': ('Electric Wizard',
  'See You In Hell',
  '2017-09-08')}

In [9]:
tracks_counter = 0
new_tracks = []

for album_id in new_albums_dict:
    next_request = spotify_request_url + 'albums/'+album_id+'/tracks?limit=50&access_token=' + access_token.strip()
    tracks_json = spotify_api.spotify_request(next_request)
    
    tracks_items = tracks_json['items']
    for item in tracks_items:
        print(item['name'] + ' -> ' + item['uri'])
        new_tracks.append(item['uri'].strip())
        tracks_counter = tracks_counter + 1
        
print('New tracks: ' + str(tracks_counter))
# GET https://api.spotify.com/v1/albums/{id}/tracks


Requesting: https://api.spotify.com/v1/albums/3eedTLY7FmfFjiV3PCaemY/tracks?limit=50&access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA
200 OK
Show Intro/Say Yeah! - Live Acoustic -> spotify:track:4QQIN8T6zZZCS4OrBgGRIj
Party Like Tomorrow Is the End of the World - Live Acoustic -> spotify:track:66Pgk0bpQaqpgOq6dOON45
Fat Girl (Thar She Blows) - Live Acoustic -> spotify:track:3BtB32aT6FLGoCuSvX1Bkt
If You Really, Really Love Me - Live Acoustic -> spotify:track:3yrKEm1fl2ccMBfnPiB8tQ
Gloryhole - Live Acoustic -> spotify:track:6XGb50TodVZ3icWNapb5Di
Bukkake Tears - Live Acoustic -> spotify:track:2u3bQzAHLdf0no9XPZdDOb
The Burden of Being Wonderful - Live Acoustic -> spotify:track:0RYql0YVIOFszv7wvXihJ8
Weenie Ride - Live Acoustic -> spotify:track:5ryMocjMGYe6dk4SZarRui
That's 

In [10]:
next_request = 'https://api.spotify.com/v1/me/playlists?access_token=' + access_token.strip()
playlists_json = spotify_api.spotify_request(next_request)

my_playlist = '5QtunQsOyazjCStL8KNz3U'

playlists_items = playlists_json['items']
for item in playlists_items:
    print(item['name'] + ' -> ' + item['id'])
    if item['name'].strip() == 'Music News (Bednar)':
        my_playlist = item['id'].strip()
        
print('\nMy playlist ID: ' + my_playlist)


Requesting: https://api.spotify.com/v1/me/playlists?access_token=BQBtYjFNafsAHDgJfpHLpFg3YKViI3VxbVVL92uva-wBzwHTPMQ69dhzqJDQj_PXmIfW1o7cLAN_l3TWY_Xc3wayaUUcSWjTUSyzQHtOZ5kEMn4emjycF5JaKisMCJOYjzG2GG4C7UaC2q_IZNhR73Vsm2NhuxZ2u7iWiOC4XM7OKdvIBjyNf5N-B5PcIy5QAChZNxn_JISAX1nVnoMMndpTJGqDAya-0366gKAa0O1rsVWguqUN0h4SaPA
200 OK
Music News (Bednar) -> 5QtunQsOyazjCStL8KNz3U
Cheerful raps -> 3IG5YeyDxr2WvmtxoABXco
Release Radar -> 37i9dQZEVXbidQSqNh7fI2
N'to - Trauma (Worakls Remix) -> 1R3pob1JtA1rkCfujHBsDV
Instrumental Madness -> 37i9dQZF1DWUk47CLxI4Uo
Discover Weekly -> 37i9dQZEVXcIleIODRmV9m
Liked from Radio -> 5XXEmY7K7glcrIHp28zJ8z

My playlist ID: 5QtunQsOyazjCStL8KNz3U


In [11]:
import requests

max_tracks_per_request = 10
url = 'https://api.spotify.com/v1/users/piotr.bednarski/playlists/' + my_playlist + '/tracks?uris='
headers = {
    'Authorization' : 'Bearer ' + access_token.strip(),
    'Content-Type' : 'application/json'
}
url_full = url
for i in range(len(new_tracks)):
    print(new_tracks[i])
    url_full = url_full + new_tracks[i] + ','
    
    if (i%max_tracks_per_request == max_tracks_per_request-1) or (i == len(new_tracks)-1):
        url_full = url_full[:-1]
        print("Request!")
        r = requests.post(url_full, headers=headers)
        print(r)
        print(r.status_code, r.reason)
        url_full = url

spotify:track:4QQIN8T6zZZCS4OrBgGRIj
spotify:track:66Pgk0bpQaqpgOq6dOON45
spotify:track:3BtB32aT6FLGoCuSvX1Bkt
spotify:track:3yrKEm1fl2ccMBfnPiB8tQ
spotify:track:6XGb50TodVZ3icWNapb5Di
spotify:track:2u3bQzAHLdf0no9XPZdDOb
spotify:track:0RYql0YVIOFszv7wvXihJ8
spotify:track:5ryMocjMGYe6dk4SZarRui
spotify:track:79bXljKuh4jKKcUe7nOTU7
spotify:track:06bf0iSYFv2aK4E1v8SS4M
Request!
<Response [201]>
201 Created
spotify:track:5flXTs1b1qB0fJJ9ww7vyL
spotify:track:1v4RB2gFd1q4pf7rUTGsVU
spotify:track:4IjBi9Vrvihl3kaSVBAzS5
spotify:track:05JkJ4YgfSbYkLlucexasZ
spotify:track:0PlVdco2zj6E60WxOBjVWW
spotify:track:5gG15aEI4zv1hohsjFBhtD
Request!
<Response [201]>
201 Created
